In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from DatabaseR import DatabaseR

# Change if not using Intel GPU
device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
print(f"Using device: {device}")

db = DatabaseR("fuel_prices.db")
fuel_type = "E10"
df = db.fetch_average_price(fuel_type=fuel_type, interval="D")

df["timestamp"] = pd.to_datetime(df["timestamp"])
df_pivot = df.pivot_table(index="timestamp", columns="station_code", values="price")

# Closest n stations
predict_station = False
if predict_station:
    target_station = "2305"
    nearby_stations = db.get_nearest_stations(target_station, fuel_type=fuel_type, count=3)
    print(nearby_stations)
    features = nearby_stations + [target_station]
    available_features = [code for code in features if code in df_pivot.columns]
    df_pivot = df_pivot[available_features]

df_pivot.bfill(inplace=True)

daily_avg = df_pivot.mean(axis=1).to_frame(name="avg_price")

scaler = MinMaxScaler()
scaled = scaler.fit_transform(daily_avg)
scaled_df = pd.DataFrame(scaled, index=daily_avg.index, columns=["avg_price"])

seq_length = 180
X, y = [], []
for i in range(len(scaled_df) - seq_length):
    X.append(scaled_df.iloc[i:i+seq_length].values)
    y.append(scaled_df.iloc[i+seq_length].values[0])

X = np.array(X)
y = np.array(y)

print("Final X shape:", X.shape)
print("Final y shape:", y.shape)

In [ ]:
class PriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = PriceDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model = LSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

best_val_loss = float('inf')
patience = 10
counter = 0

for epoch in range(100):
    model.train()
    train_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X).squeeze()
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            output = model(batch_X).squeeze()
            loss = criterion(output, batch_y)
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_avg_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping.")
            break

In [ ]:
import matplotlib.pyplot as plt

model.load_state_dict(torch.load("best_avg_model.pth"))
model.eval()

with torch.no_grad():
    inputs = torch.tensor(X, dtype=torch.float32).to(device)
    predictions = model(inputs).squeeze().cpu().numpy()

predictions_inverse = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
actual_inverse = scaler.inverse_transform(y.reshape(-1, 1)).flatten()

time_index = daily_avg.index[seq_length:]

plt.figure(figsize=(12, 6))
plt.plot(time_index, actual_inverse, label='Actual Daily Average Prices')
plt.plot(time_index, predictions_inverse, label='Predicted Daily Average Prices')
plt.title("Fuel Price Prediction")
plt.xlabel("Date")
plt.ylabel("Average Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("Date range:", daily_avg.index.min(), "to", daily_avg.index.max())
print("Predictions:", predictions_inverse[:5])
print("Actual:", actual_inverse[:5])

In [ ]:

# import datetime

future_days = 30
last_seq = torch.tensor(X[-1], dtype=torch.float32).unsqueeze(0).to(device)
future_preds = []

model.eval()
with torch.no_grad():
    for _ in range(future_days):
        output = model(last_seq)
        avg_price = output.mean().item()
        future_preds.append(avg_price)

        next_input = torch.full((1, 1, last_seq.shape[2]), avg_price, dtype=torch.float32).to(device)
        last_seq = torch.cat([last_seq[:, 1:, :], next_input], dim=1)

last_date = df_pivot.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)
future_preds_inverse = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1)).flatten()

plt.figure(figsize=(12, 6))
plt.plot(daily_avg.index[seq_length:], actual_inverse, label='Actual Daily Average')
plt.plot(daily_avg.index[seq_length:], predictions_inverse, label='Predicted (Historical)')
plt.plot(future_dates, future_preds_inverse, label=f'Forecast (Next {future_days} Days)', linestyle='--')
plt.title("Fuel Price Forecast")
plt.xlabel("Date")
plt.ylabel("Average Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame({
    "timestamp": future_dates,
    "forecast_price": future_preds_inverse
}).to_csv("30_day_forecast.csv", index=False)

In [ ]:
pd.DataFrame({
    "timestamp": df_pivot.index[-60:],
    "forecast_price": actual_inverse[-60:]
}).to_csv("prev_30_day_record.csv", index=False)